## Timing computations

+ Before proceeding further, let's review the timings measured from some preceding code snippets.

---

In [1]:
import pandas as pd

DATA_ROOT = 'bodo-examples-data/bodo-training-fundamentals'
DATA_SRC  = f's3://{DATA_ROOT}/DATA/PARQUET_010'

loading_opts = dict(storage_options=dict(anon=True))

In [2]:
%time df = pd.read_parquet(DATA_SRC, **loading_opts)

CPU times: user 11.2 s, sys: 2.54 s, total: 13.7 s
Wall time: 1min 4s


---

+ The first example we saw...
---
+ ...used the `%time` line magic command to time execution of a single line of code.
+ This is a short, simple way to time code within IPython or Jupyter notebook sessions.

In [3]:
# Embedding in a function...
import bodo, time
@bodo.jit
def compute_mean_purchase_bodo():
    start = time.time()
    DATA_ROOT = 'bodo-examples-data/bodo-training-fundamentals'
    DATA_SRC = f's3://{DATA_ROOT}/DATA/PARQUET_010'
    df = pd.read_parquet(DATA_SRC)
    avg = df['Purchase_Amount'].mean()
    print(f'Elapsed time: {time.time() - start:.3f} s')
    return avg

In [4]:
avg = compute_mean_purchase_bodo()
print(f'Average Purchase_Amount: ${avg:,.2f}')

Elapsed time: 47.231 s
Average Purchase_Amount: $184.91


---

+ The next example we saw used the function `time.time` from the `time` module.
+ Notice this gives somewhat verbose code.
---
+ The timing result is printed when the function executes.

---

### Using the time module

In [5]:
# Illustration of time.time()
import time
import pandas as pd
DATA_SRC = f's3://{DATA_ROOT}/DATA/CSV/samples_001.csv'

In [6]:
start = time.time()
df = pd.read_csv(DATA_SRC, **loading_opts)
stop = time.time()

In [7]:
print(f'Elapsed time: {stop-start:.3f} s')

Elapsed time: 69.030 s


---

+ Using `time.time` is inelegant, but simple to understand.
---
+ We start by executing setup code.
+ This includes imports and assigning `DATA_SRC`
---
+ Each invocation of `time.time` reads the system clock as a numerical value.
+ Here, we put the function calls *before* & *after* the call to Pandas `read_csv`.
---
+ The difference between those two clock readings gives the time elapsed.
+ This approach is used often in Bodo's documentation & elsewhere to illustrate timings.

---

### Using IPython magics

+ `%%time`: an Ipython *cell* magic

In [8]:
%%time
# Illustration of %%time cell magic
DATA_SRCS = [f's3://{DATA_ROOT}/DATA/CSV/samples_{k:03d}.csv'
               for k in range(1,3)]
dataframes = []
for DATA_SRC in DATA_SRCS:
    print(f'Loading {DATA_SRC}...\n')
    dataframes.append(pd.read_csv(DATA_SRC))
df = pd.concat(dataframes)

Loading s3://bodo-examples-data/bodo-training-fundamentals/DATA/CSV/samples_001.csv...

Loading s3://bodo-examples-data/bodo-training-fundamentals/DATA/CSV/samples_002.csv...

CPU times: user 9.32 s, sys: 1.4 s, total: 10.7 s
Wall time: 1min 45s


---

+ The IPython `time` *cell* magic (with *two* percent symbols) returns the time to execute a complete cell.
+ As with other cell magics internal to IPython, it must be at the very top of the cell.
+ This cell reads two individual CSV files from the S3 bucket, appends them to a list, and concatenates all the elements of the list into a single dataframe.
+ We include a `print` statement within the loop to illustrate the delay for reading each file.
---
+ On completing the execution of the *entire* cell, various numbers are output to the screen.
+ You can look up what these mean specifically; 
  + we mostly care only about `Wall` time (that is, the time as would be measured from start to finish by a clock on the wall).
+ Notice this measures the time for reading *and* the time for concatenating dataframes collectively.

---

### Using IPython magics

+ `%time`: an Ipython *line* magic

In [9]:
# Illustration of %time line magic
DATA_SRCS = [f's3://{DATA_ROOT}/DATA/CSV/samples_{k:03d}.csv'
               for k in range(1,3)]
dataframes = []
for DATA_SRC in DATA_SRCS:
    print(f'\nLoading {DATA_SRC}...\n')
    %time dataframes.append(pd.read_csv(DATA_SRC))
df = pd.concat(dataframes)


Loading s3://bodo-examples-data/bodo-training-fundamentals/DATA/CSV/samples_001.csv...

CPU times: user 4.89 s, sys: 656 ms, total: 5.55 s
Wall time: 1min 7s

Loading s3://bodo-examples-data/bodo-training-fundamentals/DATA/CSV/samples_002.csv...

CPU times: user 4.61 s, sys: 619 ms, total: 5.23 s
Wall time: 1min 10s


---

+ As an alternative, the IPython `time` *line* magic (with *one* percent symbol) returns the time to execute a *single line of a cell*.
---
+ This cell repeats the preceding computation.
+ This time, because the `time` magic is embedded within the loop, it prints to the screen within each iteration.
+ The output is rather messy but it is quick to add `percent time` to an individual line to get its execution time separate from the other code required for set up.

---

### Summary

+ Within generic Python functions:
  1. `time.time`: for timing tailored blocks of code

+ Within IPython/Jupyter sessions:
  2. `%%time` for timing *cells*
  3. `%time` for timing *lines*

---

+ To summarise, we'll use all these approaches to timing recognising their relative tradeoffs
  + verbose code input vs. verbose code output
  + fine-grained line-by-line timing vs. coarse-grained timing of an entire cell
+ Of course, developers use more sophisticated profiling tools; these are the simple enough for the moment.

---